# Vanilla Scikit-Learn Conversion to ONNX

Source: [entbappy/ONNX-Open-Neural-Network-Exchange/ML_example](https://github.com/entbappy/ONNX-Open-Neural-Network-Exchange/tree/master/ML_example).

## Train and Save Pickle

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib

iris = load_iris()
X,y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
joblib.dump(clf, 'output/model_sklearn.pkl', compress=9)

['output/model_sklearn.pkl']

## Convert to ONNX

In [4]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import joblib

cls = joblib.load('output/model_sklearn.pkl')
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(cls, initial_types=initial_type)

with open('output/model_sklearn.onnx', 'wb') as f:
    f.write(onx.SerializeToString())

## Visualize

In [5]:
import netron

In [6]:
# Start visualization server
# Browser opens to visualize the ONNX model
# http://localhost:8080/
# We can click as see what's inside!
netron.start('output/model_sklearn.onnx')

Serving 'output/model_sklearn.onnx' at http://localhost:8080


('localhost', 8080)

In [7]:
# Stop visualization server
netron.stop()

Stopping http://localhost:8080


## Inference

If we want to use the pickle to run the inference, we need the `sklearn` library. However, if we have exported to ONNX and we want to use the ONNX model for inference, we don't need the `sklearn` library, just ONNX (runtime).

In [9]:
import onnxruntime as rt
import numpy as np

data = np.array([[4.5,4.9,5.1,5.4],[1.5,2.9,3.1,1.4],[7.5,6.9,8.1,6.4]])

sess = rt.InferenceSession("output/model_sklearn.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

pred_onnx = sess.run([label_name], {input_name: data.astype(np.float32)})[0]
print(pred_onnx)

[2 1 2]
